In [39]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("tum-nlp/bert-hateXplain")
model = AutoModelForSequenceClassification.from_pretrained("tum-nlp/bert-hateXplain")
hate_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

df = pd.read_csv("../mdatasci/compsci-760/project/supervision_test10_threads.csv")
df = df.rename(columns={"is_hate": "is_hate_legacy", "hate_label": "hate_label_legacy"})

Device set to use mps:0


In [ ]:
def get_token_length(text: str) -> int:
    try:
        encoded_input_fixed = tokenizer(
            text, max_length=512, truncation=True, return_tensors="pt"
        )
        return encoded_input_fixed["input_ids"].shape[1]
    except Exception as e:
        print(text)
        print(e)
        return 0


df = df[df["body"].notnull()].copy()
df["token_len"] = df["body"].map(get_token_length)

512

In [ ]:
def classify_text(text):
    clean_text = str(text).replace("\n", " ")
    result = hate_classifier(clean_text)
    return result[0]["label"], result[0]["score"]


valid_df = df[
    (df["body"] != "[removed]") & (df["body"] != "[deleted]") & (df["token_len"] < 512)
].copy()

print(f"Processing {len(valid_df)} posts.")

tqdm.pandas(desc="Classifying posts")

print(f"Shape before: {valid_df.shape}")

classification_results = valid_df["body"].progress_apply(classify_text)
valid_df["hate_label"] = [result[0] for result in classification_results]
valid_df["hate_score"] = [result[1] for result in classification_results]

print(f"Shape after: {valid_df.shape}")
print(valid_df["hate_label"].value_counts())
valid_df = valid_df.sort_values("hate_score")
valid_df.to_csv("supervision_test10_threads_hatexplain.csv")

Processing 918 posts.
Shape before: (918, 11)


Classifying posts: 100%|██████████| 918/918 [00:04<00:00, 227.31it/s]

Shape after: (918, 12)
hate_label
non-toxic    811
toxic        107
Name: count, dtype: int64


## Scratchpad


In [12]:
import polars as pl

df = (
    pl.read_csv(
        "/Users/sujay/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Course Documents/COMPSCI 760/Project/COMPSCI 760 - Group Project/Sample Data/addNew/retrain_validation10.csv"
    )
    .drop_nans()
    .drop_nulls()
)

In [ ]:
print(
    df.group_by("subreddit")
    .agg(
        pl.col("toxicity_probability_self").mean().alias("avg_toxicity"),
        pl.len().alias("comment_count"),
    )
    .top_k(by="avg_toxicity", k=1000)
    .top_k(by="comment_count", k=10)
    .sort("avg_toxicity", descending=True)
)

shape: (10, 3)
┌───────────────────┬──────────────┬───────────────┐
│ subreddit         ┆ avg_toxicity ┆ comment_count │
│ ---               ┆ ---          ┆ ---           │
│ str               ┆ f64          ┆ u32           │
╞═══════════════════╪══════════════╪═══════════════╡
│ The_Donald        ┆ 0.33245      ┆ 21866         │
│ news              ┆ 0.315804     ┆ 11310         │
│ todayilearned     ┆ 0.313934     ┆ 10129         │
│ worldnews         ┆ 0.304131     ┆ 11230         │
│ politics          ┆ 0.299645     ┆ 39005         │
│ pics              ┆ 0.295496     ┆ 7349          │
│ gifs              ┆ 0.295433     ┆ 7733          │
│ mildlyinteresting ┆ 0.294152     ┆ 3453          │
│ videos            ┆ 0.291407     ┆ 7057          │
│ AskReddit         ┆ 0.28538      ┆ 76132         │
└───────────────────┴──────────────┴───────────────┘
